<a href="https://colab.research.google.com/github/dohyeonkim2526/BigContest/blob/master/BigCon0924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import numpy as np
import pandas as pd

base_dir='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/문제데이터_update_0818/제공데이터'
sales_data='2019년 실적데이터.xlsx'
sales_dir=os.path.join(base_dir, sales_data)

base_dir_2='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/문제데이터_update_0818/평가데이터'
pred_data='2020년 6월 평가데이터.xlsx'
pred_dir=os.path.join(base_dir_2, pred_data)

base_dir_3='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/문제데이터_update_0818/제공데이터'
rate_data ='시청률 데이터.xlsx'
rate_dir=os.path.join(base_dir_3, rate_data)

# [예측 데이터]

In [4]:
df_preds=pd.read_excel(pred_dir,header=1)
df_preds  # 예측 상품 데이터프레임

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN


In [5]:
df_preds=df_preds.drop(df_preds[(df_preds['판매단가']==0)].index) # 예측 상품 판매가 0원인 것은 예측에서 제외
df_preds=df_preds.reset_index(drop=True)

In [6]:
# 예측 상품의 마더코드 리스트 생성(pred_code)

pred_code=[]
for code in df_preds['마더코드']:
  if code not in pred_code:
    pred_code.append(code)

pred_code.sort()

# [ 2019 판매 데이터 ]

In [190]:
df_sales=pd.read_excel(sales_dir,header=1, na_values='NaN')  #원본 제공데이터, 가져와서 다시 구하기

In [191]:
del_idx = df_sales[df_sales['상품군']=='무형'].index
del_1 = df_sales.drop(del_idx)

In [192]:
del_1.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액       1993
dtype: int64

In [193]:
del_1['year']=del_1['방송일시'].dt.year
del_1['month']=del_1['방송일시'].dt.month
del_1['day']=del_1['방송일시'].dt.day_name()
del_1['hour']=del_1['방송일시'].dt.hour
del_1['min']=del_1['방송일시'].dt.minute

In [194]:
del_1=del_1.reset_index(drop=True)
del_1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,year,month,day,hour,min
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,1,Tuesday,6,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,1,Tuesday,6,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,1,Tuesday,6,20
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,1,Tuesday,6,20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019,1,Tuesday,6,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,2019,12,Tuesday,23,40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,2020,1,Wednesday,0,0
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020,1,Wednesday,0,0
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020,1,Wednesday,0,0


In [195]:
sat = []
for i in range(del_1.shape[0]):
  row = del_1.iloc[i]
  row_day = row['day']
  row_hour = row['hour']
  row_min = row['min']
  if row_day == 'Saturday':
    if row_hour == 18:
      if row_min == 0 or row_min == 20:
        sat.append(i)
  else:
    continue

In [196]:
for i in range(del_1.shape[0]):
  if i in sat:
    del_1=del_1.drop(i,axis=0)

In [197]:
del_sat=del_1.reset_index(drop=True)

In [198]:
#del_sat이 제대로 이루어졌는지 확인

test=[]

for i in range(del_sat.shape[0]):
  row=del_sat.iloc[i]
  row_day=row['day']
  row_hour=row['hour']
  if row_day == 'Saturday':
    if row_hour == 18:
      test.append(i)
  else:
    continue

In [199]:
del_test=[]

for day in test:
  del_sat_test=del_sat.iloc[day]
  del_test.append(del_sat_test)

testDF=pd.DataFrame(del_test)

In [200]:
for i in range(testDF.shape[0]):
  test_row=testDF.iloc[i]
  test_day=test_row['day']
  test_hour=test_row['hour']
  test_min=test_row['min']
  if test_day == 'Saturday':
    if test_hour == 18:
      if test_min == 0 or test_min == 20:
        print("It has not been completely removed.")
  else:
    continue

#testDF # 전체 데이터프레임에서 'Saturday' 18시 기준으로 (18:00 & 18:20) 데이터가 모두 지워졌음을 확인할 수 있다.

In [201]:
del_sat['date']=del_sat['방송일시'].dt.date

In [202]:
del_sat=del_sat.drop(['hour','min'],axis=1) #토요일 18-18:20분을 제거한 데이터프레임
del_sat['방송일시']=del_sat['방송일시'].dt.time

In [203]:
del_sat.rename(columns={"date": "방송날짜","방송일시":"방송시간"},inplace=True)

In [204]:
del_sat=del_sat[['방송날짜','방송시간','노출(분)','마더코드','상품코드','상품명','상품군','판매단가','취급액','year','month','day']]

In [271]:
perform=del_sat

In [272]:
# 2019 판매 상품의 마더코드 리스트 생성

sales_code=[]
for code in perform['마더코드']:
  if code not in sales_code:
    sales_code.append(code)

sales_code.sort()

## [ 2019 시청률 데이터 ]

In [237]:
import datetime as dt

In [238]:
rating=pd.read_excel(rate_dir,header=1)

In [239]:
rating.drop(rating.tail(1).index,inplace=True)
idx_list = rating['시간대'].tolist()
print(idx_list)

['02:00', '02:01', '02:02', '02:03', '02:04', '02:05', '02:06', '02:07', '02:08', '02:09', '02:10', '02:11', '02:12', '02:13', '02:14', '02:15', '02:16', '02:17', '02:18', '02:19', '02:20', '02:21', '02:22', '02:23', '02:24', '02:25', '02:26', '02:27', '02:28', '02:29', '02:30', '02:31', '02:32', '02:33', '02:34', '02:35', '02:36', '02:37', '02:38', '02:39', '02:40', '02:41', '02:42', '02:43', '02:44', '02:45', '02:46', '02:47', '02:48', '02:49', '02:50', '02:51', '02:52', '02:53', '02:54', '02:55', '02:56', '02:57', '02:58', '02:59', '03:00', '03:01', '03:02', '03:03', '03:04', '03:05', '03:06', '03:07', '03:08', '03:09', '03:10', '03:11', '03:12', '03:13', '03:14', '03:15', '03:16', '03:17', '03:18', '03:19', '03:20', '03:21', '03:22', '03:23', '03:24', '03:25', '03:26', '03:27', '03:28', '03:29', '03:30', '03:31', '03:32', '03:33', '03:34', '03:35', '03:36', '03:37', '03:38', '03:39', '03:40', '03:41', '03:42', '03:43', '03:44', '03:45', '03:46', '03:47', '03:48', '03:49', '03:50', 

In [240]:
del rating['2019-01-01 to 2019-12-31']

In [241]:
col_names = rating.columns.tolist()
col_names = col_names[1:]

In [242]:
new_idx = []
for col in col_names:
  for date in idx_list:
    hap = col + " " + date
    new_idx.append(hap)

In [243]:
new_idx = pd.DataFrame(new_idx)
len(new_idx)

525600

In [244]:
df_lst = []
for i in col_names:
  df_lst.append(rating[i])

new_df = pd.concat(df_lst, ignore_index=True)

In [245]:
final_df = pd.concat([new_idx, new_df], axis=1)
final_df.columns = ['date','rate']
#final_df = final_df.set_index('date')

In [246]:
final_df['date'] = pd.to_datetime(final_df['date'])

In [247]:
final_df['방송시간']=final_df['date'].dt.time
final_df['date']=final_df['date'].dt.date

In [248]:
final_df.rename(columns={"date":"방송날짜"},inplace=True)

In [249]:
final_df=final_df[['방송날짜','방송시간','rate']]

In [250]:
final_df

,방송날짜,방송시간,rate
0,2019-01-01,02:00:00,0.0
1,2019-01-01,02:01:00,0.0
2,2019-01-01,02:02:00,0.0
3,2019-01-01,02:03:00,0.0
4,2019-01-01,02:04:00,0.0
...,...,...,...
525595,2019-12-31,01:55:00,0.0
525596,2019-12-31,01:56:00,0.0
525597,2019-12-31,01:57:00,0.0
525598,2019-12-31,01:58:00,0.0


In [275]:
perform.rename(columns={"date": "방송날짜","방송일시":"방송시간"},inplace=True)

In [277]:
perform[:5] #상품별로 마더코드 다른 점 이용

,방송날짜,방송시간,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,year,month,day
0,2019-01-01,06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,1,Tuesday
1,2019-01-01,06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,1,Tuesday
2,2019-01-01,06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,1,Tuesday
3,2019-01-01,06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,1,Tuesday
4,2019-01-01,06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019,1,Tuesday


In [227]:
#group=del_sat.groupby(['방송날짜','마더코드'],as_index=False)['노출(분)']
#group.sum()

In [260]:
test=del_sat.iloc[0]
test=test['방송시간']
test2=del_sat.iloc[2]
test2=test2['방송시간']
print(test)
print(test2)

06:00:00
06:20:00


## [2019 판매/예측 마더코드의 비교→예측 데이터에서 과거 실적이 없는 상품들을 찾아낸다.]

In [273]:
no_sales_code=[]  # 예측 데이터 기준: 과거 실적이 없는 마더코드 리스트
for pred in pred_code:
  if pred not in sales_code:
    no_sales_code.append(pred)

no_sales_code.sort()

In [274]:
df_only_in_preds=pd.DataFrame()

for code in no_sales_code:
  code_preds=df_preds[(df_preds['마더코드']==code)]
  df_only_in_preds=pd.concat([df_only_in_preds,code_preds])

df_only_in_preds=df_only_in_preds.reset_index(drop=True)

In [278]:
df_only_in_preds  # 과거 실적이 없는 예측 상품 데이터프레임

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-25 08:20:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
1,2020-06-25 08:20:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
2,2020-06-25 08:40:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
3,2020-06-25 08:40:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
4,2020-06-25 09:00:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
...,...,...,...,...,...,...,...,...
1192,2020-06-11 11:20:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1193,2020-06-11 11:40:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN
1194,2020-06-11 11:40:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1195,2020-06-11 12:00:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN




## [ 과거 실적이 없는 경우 분석 ]



* 예측 데이터 기준 과거 실적이 없는 경우 → 19년도 판매 데이터에 해당 제품의 마더 코드가 없음을 의미한다.

In [279]:
df_only_in_preds

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-25 08:20:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
1,2020-06-25 08:20:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
2,2020-06-25 08:40:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
3,2020-06-25 08:40:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
4,2020-06-25 09:00:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
...,...,...,...,...,...,...,...,...
1192,2020-06-11 11:20:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1193,2020-06-11 11:40:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN
1194,2020-06-11 11:40:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1195,2020-06-11 12:00:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN


In [280]:
product_group=df_preds['상품군'].unique()
product_group

array(['의류', '속옷', '이미용', '주방', '농수축', '가전', '생활용품', '가구', '잡화', '건강기능',
       '침구'], dtype=object)

## [19년도 데이터 분석 - 판매량 기준]

* 주문량에 대한 new column 생성 + 취급액 0원인 것은 주문량인 0인 것이므로 제거

In [281]:
def cal_sales(x,y):
  return round(x/y)

perform['주문량']=cal_sales(perform['취급액'],perform['판매단가'])
perform=perform.dropna(subset=['취급액'])
perform_1=perform.sort_values(by=['주문량'],ascending=False)

In [282]:
perform_1

,방송날짜,방송시간,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,year,month,day,주문량
34965,2019-12-07,01:40:00,17.166667,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,322009000.0,2019,12,Saturday,8070.0
32500,2019-11-10,17:00:00,20.000000,100253,200864,안동간고등어 20팩,농수축,30900,199541000.0,2019,11,Sunday,6458.0
36465,2019-12-21,17:40:00,20.000000,100418,201322,코몽트 덤블 플리스 자켓 2종,의류,39900,223492000.0,2019,12,Saturday,5601.0
36937,2019-12-27,00:00:00,20.000000,100323,201025,피시원 국내산 햇 손질문어 7팩,농수축,49900,270284000.0,2019,12,Friday,5417.0
33648,2019-11-22,19:00:00,20.000000,100317,201665,국내산 손질 햇 갑오징어 9팩,농수축,49900,241079000.0,2019,11,Friday,4831.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29301,2019-10-08,10:00:00,NaN,100787,202311,[루이띠에] 18k 인피니티 러브 귀걸이,잡화,370000,532000.0,2019,10,Tuesday,1.0
20535,2019-07-16,00:40:00,20.000000,100153,200558,일시불 LG전자 휴대용 퓨리케어 미니 공청기,가전,309000,421000.0,2019,7,Tuesday,1.0
29304,2019-10-08,10:30:00,NaN,100787,202315,[루이띠에] 18k 인피니티러브 목걸이 팔찌 세트,잡화,940000,1278000.0,2019,10,Tuesday,1.0
20533,2019-07-16,00:20:00,20.000000,100153,200558,일시불 LG전자 휴대용 퓨리케어 미니 공청기,가전,309000,409000.0,2019,7,Tuesday,1.0


In [283]:
perform_1.groupby('상품군')['상품군'].count() #(19년도 판매량 순위) 주방 > 가전 > 의류 > 속옷 > 농수축> 잡화 > 가구 > 기타

상품군
가구      2302
가전      5163
건강기능     783
농수축     3875
생활용품    2762
속옷      3907
의류      4331
이미용     1304
잡화      3694
주방      6571
침구       664
Name: 상품군, dtype: int64

## [19년도 데이터 분석 - 방송시간 기준]




In [284]:
perform_2=perform[perform['노출(분)'].notnull()]

In [285]:
perform_2_NaN=perform[perform['노출(분)'].isnull()]
uni=perform_2_NaN['상품명'].unique()

In [286]:
check_uni=[] # 한 상품명에 대하여 노출(분)이 0인 것에 대해 예외가 존재하는지 확인하기 위한 리스트 생성

for name in perform_2:
  if name in uni:
    check_uni.append(name)
  check_uni.sort()

check_uni

[]

* 노출(분)이 0인 상품의 경우, 노출(분)이 0이 아닌 예외적인 경우가 존재하지 않음을 알 수 있다.

In [287]:
perform_2_NaN

,방송날짜,방송시간,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,year,month,day,주문량
1,2019-01-01,06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,1,Tuesday,110.0
3,2019-01-01,06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,1,Tuesday,174.0
5,2019-01-01,06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,2019,1,Tuesday,234.0
26,2019-01-01,14:00:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,20841000.0,2019,1,Tuesday,175.0
28,2019-01-01,14:30:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,47294000.0,2019,1,Tuesday,397.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37343,2019-12-31,23:40:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,2019,12,Tuesday,15.0
37344,2019-12-31,23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,2019,12,Tuesday,69.0
37346,2020-01-01,00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020,1,Wednesday,621.0
37347,2020-01-01,00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020,1,Wednesday,87.0


In [288]:
count=perform_2_NaN.groupby('상품명')['주문량'].sum()
count=count.to_frame()
count

,주문량
상품명,
[가이거] 블랙 에디션 다이아몬드 워치(남성용),185.0
[맛있는 제주]손질 생선 3대세트+흑돼지구이(광어+갈치+고등어,461.0
메디아글램 카이만엠보 소가죽 체인숄더백 (+클러치백+폼폼이),728.0
메디아글램 카이만엠보 소가죽 토트백 (+클러치백+폼폼이),558.0
(1등급)221L_딤채김치냉장고,236.0
...,...
효재 자수 광목 워싱 침구세트 퀸,772.0
효재 자수 광목 워싱 침구세트 킹,1124.0
효재 화원 자수 광목워싱 침구세트 슈퍼싱글,135.0
